### Dependencies

In [1]:
import cv2 as cv
import numpy as np

In [2]:
DEFAULT_FILTER = cv.COLOR_BGR2GRAY

In [ ]:
WIN_NAME = "img2gray"

def im2gray(path: str) -> None:
    try:
        src = cv.imread(path)
        
        if src is None:  
            raise FileNotFoundError(f"Image not found at path: {path}")
        
        img = cv.cvtColor(src, DEFAULT_FILTER)
        cv.imshow(WIN_NAME, img)
        cv.waitKey(0)
        cv.imwrite(f'{path}-2gray.png', img)
    except FileNotFoundError as e:
        print(f"Something went wrong: {e}")

In [3]:
SRC = "ford.png"

im2gray(SRC)

In [2]:
class HSVColor:
    def __init__(self, lower: np.ndarray, upper: np.ndarray):
        self.lower = lower
        self.upper = upper

In [ ]:
blueMask= HSVColor(np.array([100, 150, 50]), np.array([140, 255, 255]))

# ip: http://esp-ip:81/stream
cam = cv.VideoCapture(0)

while True:
    if not cam.isOpened():
        print(f'Camera not found')
        exit()

    _, frame = cam.read()
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HLS)
    filterMask = cv.inRange(hsv, blueMask.lower, blueMask.upper)

    cv.imshow('HSV result', frame)
    contours, _ = cv.findContours(filterMask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    if cv.waitKey(1) == ord('q'):
        break

cam.release()
cv.destroyAllWindows()